# Customer Sales Representative
    Hey I am a customer sales representative. My only work here is to entertain our beloved clients and in results I get a pay raise. (Party) 

In [7]:
import pymongo

myclient = pymongo.MongoClient("mongodb://192.168.18.14:27017,192.168.18.14:27018,192.168.18.14:27019/?replicaSet=mongodb-replicaset")

print(myclient.list_database_names())

In [8]:
print(myclient.list_database_names())

ServerSelectionTimeoutError: mongoset3:27019: [Errno -2] Name or service not known,mongoset1:27017: [Errno -2] Name or service not known,mongoset2:27018: [Errno -2] Name or service not known, Timeout: 30s, Topology Description: <TopologyDescription id: 60e18deee9a17c22678c8c60, topology_type: ReplicaSetNoPrimary, servers: [<ServerDescription ('mongoset1', 27017) server_type: Unknown, rtt: None, error=AutoReconnect('mongoset1:27017: [Errno -2] Name or service not known',)>, <ServerDescription ('mongoset2', 27018) server_type: Unknown, rtt: None, error=AutoReconnect('mongoset2:27018: [Errno -2] Name or service not known',)>, <ServerDescription ('mongoset3', 27019) server_type: Unknown, rtt: None, error=AutoReconnect('mongoset3:27019: [Errno -2] Name or service not known',)>]>

In [ ]:
from pymongo import MongoClient
import pymongo


connectionString = "mongodb://mongo:27017,192.168.18.14:27018,192.168.18.14:27019/magic_shop?replicaSet=mongodb-replicaset";
client           = MongoClient(connectionString)
db               = client.magic_shop        # test is my database
col              = db.req                   # Here spam is my collection

# Listening to the Insert Stream

In [ ]:
import socket
machine_name = socket.gethostname()
from datetime import datetime
import time

def processing_request(request):
    '''
    Description:
        Get the string, takes its length and run a loop 10 times of string length and set the 
        response to processed.
        
    Input:
        request  (str) : Random String
        
    Output:
        Set value back to the mongo db
            response: Processed
    '''
    for i in range(0,len(request)*10):
        pass
    
    

try:
    with db.req.watch([{'$match': {'operationType': 'insert'}}]) as stream:
        print("Catching Insert Triggers")
        for values in stream:
            request_id = values['fullDocument']['request_id']
            request    = values['fullDocument']['request']
            myquery    = { "request_id": request_id }
            
            # Checking if not processed by other replica (Blocking System)
            if len(list(col.find({ 'request_id': request_id, 'CSR_NAME' : { "$exists": False}}))):
                newvalues  = { "$set": { "CSR_NAME": machine_name, "response": "Processing", "processing_time":datetime.today()} }

                # CSR Responding to the client
                col.update_one(myquery, newvalues)
                print(request_id)
                print(f"Processing {request_id}")
                # Now Processing Request
                processing_request(request)
                # Updating that work is done

                myquery   = { "request_id": request_id }
                newvalues = { "$set": {"response": "Request Processed Have a nice Day sir!", 'response_time':datetime.today()} }
                # CSR Responding to the client
                col.update_one(myquery, newvalues)
            
except pymongo.errors.PyMongoError:
    # The ChangeStream encountered an unrecoverable error or the
    # resume attempt failed to recreate the cursor.
    logging.error('...')